In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from geopy.geocoders import Nominatim
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, accuracy_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [79]:
df = pd.read_csv('SaltyAnimators Salary Collection.csv',encoding='utf-8')
df.describe()


,RATE,Role,Seniority,Location,Years,Company,Gender,Union
count,2302,2301,2229,2281,2281,994,2238,1737
unique,2063,1014,287,587,293,527,4,189
top,60000.00,3D Animator,Junior,Vancouver,3,Freelance,Female,No
freq,8,243,447,218,233,39,1251,1082


In [80]:
print(df.shape)
df.isnull().sum()

(2302, 8)


RATE            0
Role            1
Seniority      73
Location       21
Years          21
Company      1308
Gender         64
Union         565
dtype: int64

In [81]:
#Basic cleanings
for c in df:
    df[c] = df[c].str.lower()
    df[c].str.replace(',','')
    df[c].str.replace(' ','')


In [82]:
#Deal with Gender
tmp = []
for index, row in df.iterrows():
    if 'male' in str(row['Gender']):
        tmp.append(1)
    elif 'female' in str(row['Gender']):
        tmp.append(0)
    else:
        df.drop(index, inplace=True)
df['Gender'] = tmp  

In [83]:
#Deal with Company
tmp = []
for data in df['Company']:
    if 'freelance' in str(data):
        tmp.append(1)
    else:
        tmp.append(0)

df['Company'] = tmp

In [84]:
#Deal with Union
tmp = []
for data in df['Union']:
    if 'no' in str(data):
        tmp.append(1)
    else:
        tmp.append(0)

df['Union'] = tmp

In [85]:
#Deal with Years
tmp = []
for index, row in df.iterrows():
    year = ''.join(i for i in str(row['Years']) if i.isdigit())
    
    if len(year)== 0:
        df.drop(index, inplace=True)
    else:
        tmp.append(year)

df['Years'] = tmp

In [86]:
#Deal with Seniority
tmp = []
for index, row in df.iterrows():
    if 'director' in str(row['Seniority']):
        tmp.append(0)
    elif 'supervisor' in str(row['Seniority']):
        tmp.append(1) 
    elif 'senior' in str(row['Seniority']) or 'sr' in str(row['Seniority']):
        tmp.append(2) 
    elif 'mid' in str(row['Seniority']) or 'junior' in str(row['Seniority']) or 'Intermediate' in str(row['Seniority']) or 'jr' in str(row['Seniority']):
        tmp.append(3)
    elif len(str(row['Seniority']))== 0:
        df.drop(index, inplace=True)
    else:
        df.drop(index, inplace=True)

df['Seniority'] = tmp

In [87]:
#Deal with Rate
tmp = []
for index, row in df.iterrows():
    rate = ''.join(i for i in str(row['RATE']) if i.isdigit())
    if len(rate)>7:
        df.drop(index, inplace=True)
    elif len(rate)== 0:
        df.drop(index, inplace=True)
    else:
        tmp.append(rate)


df['RATE'] = tmp

In [88]:
ohe = OneHotEncoder()
df.dropna(inplace = True)

In [89]:
#Deal with Location
# tmp = []
# loc = Nominatim(user_agent="GetLoc")
# geoLoc = Nominatim(user_agent="GetLoc")

# for index, row in df.iterrows():
#     getLoc = loc.geocode(str(row['Location']))
#     tmp.append(getLoc)

# df['Location'] = tmp
dummy_location = pd.get_dummies(df['Location'])

df = pd.merge(
    left=df,
    right=dummy_location,
    left_index=True,
    right_index=True,
)


In [90]:
#Deal with Role
dummy_role = pd.get_dummies(df['Role'])
df = pd.merge(
    left=df,
    right=dummy_role,
    left_index=True,
    right_index=True,
)

df.drop(['Location', 'Role'], axis=1, inplace=True)

In [91]:
df.dropna(thresh=1,inplace = True)
df["RATE"] = df["RATE"].astype("float")
df["Seniority"] = df["Seniority"].astype("float")
df["Years"] = df["Years"].astype("float")
df["Company"] = df["Company"].astype("float")
df["Gender"] = df["Gender"].astype("float")
df["Union"] = df["Union"].astype("float")



y = df['RATE']
X = df.drop(df.columns[[0]], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
scaler = StandardScaler()
X_std = scaler.fit_transform(X_train)

regressor = LinearRegression()
regressor.fit(X_train,y_train) 

y_pred = regressor.predict(X_test) 
print("MAE: ",mean_absolute_error(y_pred,y_test))

MAE:  13991720861218.537


In [95]:
regressor_RF = RandomForestRegressor(n_estimators=10000, n_jobs=-1, max_depth=50,random_state=666)
regressor_RF.fit(X,y)
y_pred = regressor_RF.predict(X_test) 
print("MAE: ",mean_absolute_error(y_pred,y_test))

In [93]:
import xgboost as xgb
xgb_reg = xgb.XGBRegressor(max_depth=50, n_estimators=1000, n_jobs=2,
                           objectvie='reg:squarederror', booster='gbtree',
                           random_state=666, learning_rate=0.05)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test) 
print("MAE: ",mean_absolute_error(y_pred,y_test))


[11:07:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "objectvie" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


MAE:  589951.9659575038


In [94]:
# from sklearn.preprocessing import PolynomialFeatures
# poly_reg = PolynomialFeatures(degree=2)
# X_poly = poly_reg.fit_transform(X)
# lin_reg_2 = LinearRegression()
# lin_reg_2.fit(X_poly,y)
# y_pred = lin_reg_2.predict(X_test) 
# print("MAE: ",mean_absolute_error(y_pred,y_test))